# Main

In [ ]:
# Data manipulation
import pandas as pd # data manipulation and dataframes

# Datetime
from datetime import datetime

In [ ]:
urls = get_product_urls('https://www.dia.es/')

In [ ]:
# Wanted information of each product
scraping_result = {'supermarket': [],
                'category': [],
                'subcategory': [],
                'name': [], 
                'price': [], 
                'reference_price': [],
                'reference_unit': [],
                'insert_date': []}


scraping_result = scrap_products(scraping_result, urls)

# Export scraping result to csv
df = pd.DataFrame(scraping_result)
today_date = datetime.today().strftime('%Y-%m-%d %H-%M-%S')
df.to_csv(f'../scrap({today_date}).csv', index = True, sep = ',')

# Checks performance
print(f'Scraped products: {len(scraping_result["name"])} of {total}')